In [44]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException        


In [90]:
driver = webdriver.Chrome()

In [1]:
USERNAME = "your_username_here"

In [102]:
driver.get("https://data.typeracer.com/pit/race_history?user=%s"%USERNAME)

In [92]:
#Go to the site and accept the cookies if you have to

In [93]:
def findByClass(element, class_name, multiple=False):
    if(multiple):
        return element.find_elements_by_class_name(class_name)
    return element.find_element_by_class_name(class_name)
def findByTag(element, tag_name, multiple=False):
    if(multiple):
        return element.find_elements_by_tag_name(tag_name)
    return element.find_element_by_tag_name(tag_name)

def findByPartialText(element, text, multiple=False):
    return element.find_element_by_xpath(containsTextXpath(text))
    
def containsTextXpath(text):
    return "//*[contains(text(), \"%s\")]"%text
def get_table_data(driver):
    history = []
    table = findByClass(driver, 'scoresTable')
    tableData = findByTag(table, 'tbody')
    rows = findByTag(tableData, 'tr', multiple=True)
    print(len(rows)-1, ' rows found in the table... \n')
    #Finding the headers of the table
    header_row = rows[0]
    header_items = findByTag(header_row,'th',multiple=True)
    headers = list(map(lambda x:x.text.strip(), header_items))[:-1]
    print('Headers\n', headers)
    print('\n')
    print('Data')
    for row in rows[1:]:
        table_items = findByTag(row,'td',multiple=True)
        tds = list(map(lambda x:x.text.strip(), table_items))[:-1]
        print(tds)
        assert len(headers)==len(tds), 'Headers and Data length mismatch' 
        hist_entry = dict()
        for key, val in zip(headers, tds):
            if key=='Date':
                val = val.replace(',', ' ')
            hist_entry[key] = val
        history.append(hist_entry)
    return history

def checkExistsByXpath(element, xpath):
    try:
        element.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True
def lastPage(driver):
    check = "load older results"
    xpath = containsTextXpath(check)
    return not checkExistsByXpath(driver, xpath)

In [95]:
all_history = []
while(not lastPage(driver)):
    history = get_table_data(driver)
    all_history+=history
    next_page = findByPartialText(driver, "older results")
    next_page.click()
#Fetching the last pages data!
history = get_table_data(driver)
all_history+=history

20  rows found in the table... 

Headers
 ['Race #', 'Speed', 'Accuracy', 'Points', 'Place', 'Date']


Data
['139', '60 WPM', '95.7%', '72', '2/5', 'May 21, 2019']
['138', '76 WPM', '97.8%', '74', '1/5', 'May 21, 2019']
['137', '60 WPM', '95.8%', '34', '1/3', 'May 21, 2019']
['136', '69 WPM', '97.4%', '45', '2/5', 'May 21, 2019']
['135', '53 WPM', '97.3%', '55', '2/5', 'May 21, 2019']
['134', '69 WPM', '96.5%', '41', '1/4', 'May 21, 2019']
['133', '57 WPM', '95.5%', '25', '3/5', 'May 21, 2019']
['132', '62 WPM', '97.6%', '46', '2/5', 'May 21, 2019']
['131', '68 WPM', '97.1%', '59', '2/5', 'May 21, 2019']
['130', '45 WPM', '95.7%', '45', '2/4', 'May 21, 2019']
['129', '55 WPM', '96.1%', '47', '4/5', 'May 21, 2019']
['128', '60 WPM', '95.8%', '46', '5/5', 'May 21, 2019']
['127', '70 WPM', '97.1%', '49', '2/5', 'May 21, 2019']
['126', '61 WPM', '95.7%', '59', '2/4', 'May 21, 2019']
['125', '53 WPM', '96.4%', '74', '5/5', 'May 21, 2019']
['124', '71 WPM', '97.2%', '80', '1/5', 'May 21, 201

['4', '45 WPM', '93.4%', '13', '1/5', 'Oct. 15, 2017']
['3', '48 WPM', '95.1%', '59', '4/5', 'Oct. 15, 2017']
['2', '53 WPM', '94.9%', '22', '3/5', 'Oct. 15, 2017']
['1', '42 WPM', '91.7%', '34', '5/5', 'Oct. 15, 2017']


In [96]:
len(all_history)

139

In [98]:
import pandas as pd
df = pd.DataFrame(all_history)

In [99]:
df.head()

,Accuracy,Date,Place,Points,Race #,Speed
0,95.7%,May 21 2019,2/5,72,139,60 WPM
1,97.8%,May 21 2019,1/5,74,138,76 WPM
2,95.8%,May 21 2019,1/3,34,137,60 WPM
3,97.4%,May 21 2019,2/5,45,136,69 WPM
4,97.3%,May 21 2019,2/5,55,135,53 WPM


In [100]:
df.to_csv('./typeracer_data.csv')